# Curso BancoEstado-CMM: Aprendizaje de Máquinas Avanzado 
**Autores:** Taco de Wolff y Felipe Tobar  

**Fecha:** 18 octubre, 2019


# Módulo 4 - Contenidos
Scikit-learn:
- Data preprocessing
- - Clean up, transformation, standardization, normalization
- - Show skewed distribution and transform to symmetric distribution (Yeo-Johnson, Box-Cox)
- - Discretization of features: KBins from numeric value to category (or to binary)
- - Imputation of missing values
- - PolynomialRegression for trend or higher-order discovery
- Model selection, validation, and evaluation (metrics)
- - Train versus test set
- - Accuracy and error measures and explain what they mean
- Clustering
- - K-means, means shifts, Hierarchical clustering
- - Maybe a tutorial?
- SVM?
- Regression and classification, again?
- - SVM, ridge/lasso regression, logistic regression